# Ch.6 영상 필터
## 6.1 컨볼루션과 블러링
- ㅋ컨볼루션 연산은 공간 영역 필터의 핵심, 대표적인 사례가 블러링
### 6.1.1 필터와 컨볼루션
- 입력픽셀, 커널(윈도우, 필터), 결과픽셀
- 블러링은 주변 요소 값들의 평균값을 반영해 전체 영상이 흐릿하게 함
- dst = cv2.filter2D(src, ddepth, kernel[, dst, anchor, delta, borderTyep])
    - ddepth : 출력 영상의 dtype
        - -1 : 입력영상과 동일
        - CV_8U, CV16U/CV16S, CV_32F, CV_64F
    - kernel : 컨볼루션 커널 (float32)
    - anchor : 커널의 기준점 (default : (-1, -1))
    - delta : 필터 적용된 결과에 추가할 값
    - borderType : 외곽 픽셀 보정 방법 지정

### 6.1.2 평균 블러링
- cv2.blur() : 커널의 크기만 지정하면 평균 커널을 생성해 블러링 적용
- dst = cv2.blur(src, ksize[, dst, anchor, borderTyep])
    - ksize : 커널의 크기
    - 나머지 인자는 cv2.filter2D와 동일

- cv2.boxFilter() : normalize 적용시 cv2.blur()와 동일, False의 경우 커널 영역의 모든 픽셀의 합을 구해 객체 추적 알고리즘에 사용 (ch.8.5.2 옵티컬 플로)
- dst = cv2.boxFilter(src, ddepth, ksize[, dst, anchor, normalize, borderTyep])
    - ddepth : 출력 영상의 dtype, -1 : 입력영상과 동일
    - normalize : 커널의 크기로 정규화 (1/ksize**2) 지정 여부 (boolean)
    - 나머지 인자는 cv2.filter2D와 동일

In [3]:
import cv2
import numpy as np

img = cv2.imread('img/girl.jpg')

kernel = np.array([[0.04, 0.04, 0.04, 0.04, 0.04],
                    [0.04, 0.04, 0.04, 0.04, 0.04],
                    [0.04, 0.04, 0.04, 0.04, 0.04],
                    [0.04, 0.04, 0.04, 0.04, 0.04],
                    [0.04, 0.04, 0.04, 0.04, 0.04]])

# 5x5 평균 필터 커널 생성
kernel = np.ones((5, 5))/5**2

# filter
blured = cv2.filter2D(img, -1, kernel)

# result
cv2.imshow('origin', img)
cv2.imshow('blured', blured)
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')

# blur
blur1 = cv2.blur(img, (10, 10))
blur2 = cv2.boxFilter(img, -1, (10, 10))

# result
merged = np.hstack((img, blur1, blur2))
cv2.imshow('merged', merged)
cv2.waitKey()
cv2.destroyAllWindows()

### 6.1.3 가우시안 블러링
- dst = cv2.GaussianBlur(src, ksize[, dst, anchor, borderTyep])
    - ksize : 커널 크기 (홀수)
    - sigmaX : X 방향 표준편차
        - 0 : auto, sigma = 0.3*((ksize - 1)*0.5 -1) + 0.8
    - sigmaY : Y 방향 표준편차 (default : sigmaX)
    - borderType : 외곽 테두리 보정 방식

- ret = cv2.getGaussianKernel(src, sigma[, kType])
    - ret : 가우시안 커널 (1차원이므로 ret*ret.T 형태로 사용(2차원으로 변형))

In [14]:
import cv2
import numpy as np

img = cv2.imread('img/gaussian_noise.jpg')

# 가우시안 커널을 직접 생성해 블러링
k1 = np.array([[1, 2, 1],
                [2, 4, 2],
                [1, 2, 1],])*(1/16)
blur1 = cv2.filter2D(img, -1, k1)


# 가우시안 커널 API
k2 = cv2.getGaussianKernel(3, 0)
blur2 = cv2.filter2D(img, -1, k2*k2.T) # k2 = [[a], [a], [a]] (3x1) / k2.T = [[a, a, a]] (1x3)
# a = [[1, 1, 1]]

# 가우시안 블러 API
blur3 = cv2.GaussianBlur(img, (3, 3), 0)

# result
print('k1: ', k1)
print('k2: ', k2*k2.T)

merged = np.hstack((img, blur1, blur2, blur3))
cv2.imshow('gaussian blur', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

k1:  [[0.0625 0.125  0.0625]
 [0.125  0.25   0.125 ]
 [0.0625 0.125  0.0625]]
k2:  [[0.25 0.25 0.25]
 [0.5  0.5  0.5 ]
 [0.25 0.25 0.25]]


### 6.1.4 미디언 블러링
- dst = cv2.medianBlur(src, ksize)
- 커널 영역 픽셀 값 중에 중간 값을 대상 픽셀의 값으로 선택하는 방법

In [19]:
import cv2
import numpy as np

img = cv2.imread('img/salt_pepper_noise.jpg')

# 미디언 블러 API
blur = cv2.medianBlur(img, 15)

# result
merged = np.hstack((img, blur))
cv2.imshow('median blur', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.1.5 바이레터럴 필터
- 블러링 필터는 대체로 잡음을 제거하는 데 효과가 있지만 경계도 흐릿해지는 문제점
- 바이레터럴 필터를 이용해 이 문제를 개선 (가우시안 필터와 경계 필터를 사용)
- 속도는 느림
- dst = cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace[, dst, borderType])
    - d : 필터의 직경(diameter), 5보다 크면 매우 느림
    - sigmaColor : 색공간 필터의 시그마 값
    - sigmaSpace : 좌표 공간의 시그마 값 (단순 사용시 sigmaColor와 sigmaSpace 동일한 값 권장, 범위는 10~150 권장)
    - 시그마 값 150 이상 지정 시 스케치 효과를 얻을 수 있음

In [23]:
import cv2
import numpy as np

img = cv2.imread('img/gaussian_noise.jpg')

# 가우시안 필터 적용
blur1 = cv2.GaussianBlur(img, (5, 5), 0)

# 바이레터럴 필터 적용
blur2 = cv2.bilateralFilter(img, 5, 250, 160)

# 결과 출력
merged = np.hstack((img, blur1, blur2))
cv2.imshow('bilateral', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
import cv2
import numpy as np

img = cv2.imread('img/gaussian_noise.jpg')

# 가우시안 필터 적용
blur1 = cv2.GaussianBlur(img, (5, 5), 0)

# 바이레터럴 필터 적용
blur2 = cv2.bilateralFilter(img, 5, 75, 160)

# 결과 출력
merged = np.hstack((img, blur1, blur2))
cv2.imshow('bilateral', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 6.2 경계 검출
### 6.2.1 기본 미분 필터
- 영상 속의 픽셀 데이터는 연속된 공간이 아니므로 이산화시켜서 근사값으로 간소화해야함 (간소 미분 공식은 아래와 같음)
    - Gx = af(x, y)/ax = f_x+1,y - f_x,y
    - Gy = af(x, y)/ay = f_x,y+1 - f_x,y
- 공식의 의미는 x축과 y축 각각의 방향에서 다음 픽셀의 값에서 현재 픽셀 값을 뺀다는 것임
- 영상에 대한 미분 연산의 컨볼루션 커널은 다음과 같음
    - Gx = [-1, 1] / Gy = [[-1], [1]]
- **gradient 방향과 edge의 방향은 서로 수직의 방향**

In [31]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 미분 커널 생성
gx_kernel = np.array([[-1, 1]])
gy_kernel = np.array([[-1], [1]])

# 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy))
cv2.imshow('edge', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.2 로버츠 교차 필터
- 대각선 방향으로 1과 -1을 배치해 사선 경계 검출 효과를 높였지만, 노이즈에 민감하고 엣지 강도가 약함
- Gx = [[1, 0], [0, -1]] / Gy = [[0, 1], [-1, 0]]

In [32]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 로버츠 커널 생성
gx_kernel = np.array([[1, 0], [0, -1]])
gy_kernel = np.array([[0, 1], [-1, 0]])

# 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
cv2.imshow('roberts cross', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.3 프리윗 필터
- 각 방향으로 차분을 세번 계산하도록 배치해 엣지 강도가 강하고 수직과 수평 엣지를 동등하게 찾음
- 대각선 검출은 약함
- Gx = [[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]] / Gy = [[-1, -1, -1], [0, 0, 0], [1, 1, 1]]

In [33]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 로버츠 커널 생성
gx_kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
gy_kernel = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])

# 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 결과 출력
merged = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
cv2.imshow('prewitt cross', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.4 소벨 필터
- 중심 픽셀의 차분 비중을 두 배로 주어 수평, 수직 대각선 경계 검출에 모두 강한 마스크 제안
- 1차 미분 마스크로 대표적
- Gx = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]] / Gy = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
- dst = cv2.Sobel(src, ddepth, dx, dy[, dst, ksize, scale, delta, borderType])
    - ddepth : 출력 영상의 dtype (-1 : 입력 영상과 동일)
    - dx, dy : 미분 차수 (0, 1, 2 중 선택, 둘 다 0일 수는 없음)
    - ksize : 커널의 크기 (1, 3, 5, 7)
    - scale : 미분에 사용할 계수
    - delta : 연산 결과에 가산할 값

In [36]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 소벨 커널 생성
gx_kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
gy_kernel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

# 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 소벨 API로 경계 검출
sobel_x = cv2.Sobel(img, -1, 1, 0, ksize=3)
sobel_y = cv2.Sobel(img, -1, 0, 1, ksize=3)

# 결과 출력
merged1 = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
merged2 = np.hstack((img, sobel_x, sobel_y, sobel_x+sobel_y))
merged = np.vstack((merged1, merged2))
cv2.imshow('sobel', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.5 샤르 필터
- 커널의 크기가 작은 경우, 또는 커널의 크기가 커도 중심에서 멀어질 수록 엣지의 방향성의 정확도가 떨어지는 단점이 있는 소벨 필터의 단점을 개선
- Gx = [[-3, 0, 3], [-10, 0, 10], [-3, 0, 3]] / Gy = [[-3, -10, -3], [0, 0, 0], [3, 10, 3]]
- dst = cv2.Scharr(src, ddepth, dx, dy[, dst, scale, delta, borderType])
    - ddepth : 출력 영상의 dtype (-1 : 입력 영상과 동일)
    - dx, dy : 미분 차수 (0, 1, 2 중 선택, 둘 다 0일 수는 없음)
    - scale : 미분에 사용할 계수
    - delta : 연산 결과에 가산할 값

In [38]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 샤르 커널 생성
gx_kernel = np.array([[-3, 0, 3], [-10, 0, 10], [-3, 0, 3]])
gy_kernel = np.array([[-3, -10, -3], [0, 0, 0], [3, 10, 3]])

# 필터 적용
edge_gx = cv2.filter2D(img, -1, gx_kernel)
edge_gy = cv2.filter2D(img, -1, gy_kernel)

# 샤르 API로 경계 검출
scharr_x = cv2.Scharr(img, -1, 1, 0)
scharr_y = cv2.Scharr(img, -1, 0, 1)

# 결과 출력
merged1 = np.hstack((img, edge_gx, edge_gy, edge_gx+edge_gy))
merged2 = np.hstack((img, scharr_x, scharr_y, scharr_x+scharr_y))
merged = np.vstack((merged1, merged2))
cv2.imshow('scharr', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6.2.6 라플라시안 필터
- 2차 미분을 적용해 경계를 더 확실히 검출할 수 있음, 대표적인 2차 미분 마스크가 라플라시안 필터
- d2f/dy2 = df(x,y+1)/dy - df(x,y+1)/dy = [f(x,y+1)-f(x,y)] - [f(x,y)-f(x,y-1)] = f(x,y+1) - 2*f(x,y) - f(x,y-1)
- kernel = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]
- dst = cv2.Laplacian(src, ddepth, dx, dy[, dst, ksize, scale, delta, borderType])
    - cv2.Sobel()과 함수 인자는 동일

In [39]:
import cv2
import numpy as np

img = cv2.imread('img/sudoku.jpg')

# 라플라시안 API로 경계 검출
edge = cv2.Laplacian(img, -1)

# 결과 출력
merged = np.hstack((img, edge))
cv2.imshow('Laplacian', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()